Dataset 1: Absenteeism at work. 

Source: UCI Machine Learning Repository 

URL: https://archive.ics.uci.edu/ml/datasets/Absenteeism+at+work


In [1]:
import requests
import zipfile
import io
import os
import pandas as pd
from sqlalchemy.engine import create_engine
import openai 
from dotenv import load_dotenv
from IPython.display import display, Markdown

url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00445/Absenteeism_at_work_AAA.zip"

# download the ZIP file
response = requests.get(url)

# extract the contents of the ZIP file
zf = zipfile.ZipFile(io.BytesIO(response.content))
df = pd.read_csv(zf.open("Absenteeism_at_work.csv"), sep=";", index_col=0)

# Replace spaces with underscores in the column names
df.columns = [c.replace(" ", "_").replace("/","_per_") for c in df.columns]

In [2]:
engine = create_engine("sqlite://")

df.to_sql("absenteeism", engine)

740

In [3]:
%load_ext sql
%sql engine

There's a new jupysql version available (0.7.4), you're running 0.7.2. To upgrade: pip install jupysql --upgrade


In [4]:
%%sql
SELECT *
FROM absenteeism
LIMIT 3

*  sqlite://
Done.


ID,Reason_for_absence,Month_of_absence,Day_of_the_week,Seasons,Transportation_expense,Distance_from_Residence_to_Work,Service_time,Age,Work_load_Average_per_day_,Hit_target,Disciplinary_failure,Education,Son,Social_drinker,Social_smoker,Pet,Weight,Height,Body_mass_index,Absenteeism_time_in_hours
11,26,7,3,1,289,36,13,33,239.554,97,0,1,2,1,0,1,90,172,30,4
36,0,7,3,1,118,13,18,50,239.554,97,1,1,1,1,0,0,98,178,31,0
3,23,7,4,1,179,51,18,38,239.554,97,0,1,0,1,0,0,89,170,31,2


## Natural language queries and their equivalent SQL commands

Note: you will need to create a `.env` file with the following parameters:

```
    OPENAI_API_KEY=<your open AI API key>
```

In [8]:
load_dotenv('.env')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [9]:
class Prompter:
    def __init__(self, gpt_model):
        if not os.environ.get("OPENAI_API_KEY"):
            raise Exception("Please set the OPENAI_API_KEY environment variable")

        openai.api_key = os.environ.get("OPENAI_API_KEY")

        self.gpt_model = gpt_model

    def prompt_model_print(self, messages: list):
        response = openai.ChatCompletion.create(model=self.gpt_model, messages=messages)
        display(Markdown(response["choices"][0]["message"]["content"]))
    
    def prompt_model_return(self, messages: list):
        response = openai.ChatCompletion.create(model=self.gpt_model, messages=messages)
        return response["choices"][0]["message"]["content"]
    

In [25]:
sample = dict(list(zip(df.columns, df.iloc[0].to_list())))

In [29]:
prompter = Prompter("gpt-4")

datagen_prompts = [
    {"role" : "system", "content" : "You are an interviewer, your goal is to hire an analyst specializing in SQL. \
     You are presented with a dataset, and you form natural language questions and their corresponding SQL queries. \
     Your goal is to test a candidate's ability to generate SQL queries from natural language questions."},
    {"role" : "user", "content" : f"Please generate a total of 18 natural language questions (with 6 'easy', 6 'medium' and 6 'hard' levels of difficulty, 'medium' questions should include 'JOIN', 'hard' questions should include using subqueries), and their corresponding SQL queries for data with columns {', '.join(df.columns)} and sample values {sample}. \
     The table is called 'absenteeism'"},
]

In [30]:
prompter.prompt_model_print(datagen_prompts)

Easy Questions:
1. How many records are in the absenteeism table?
   SELECT COUNT(*) FROM absenteeism;

2. How many unique employees are listed in the dataset?
   SELECT COUNT(DISTINCT Age) FROM absenteeism;

3. What is the average distance from residence to work?
   SELECT AVG(Distance_from_Residence_to_Work) FROM absenteeism;

4. What is the average number of hours of absenteeism?
   SELECT AVG(Absenteeism_time_in_hours) FROM absenteeism;

5. How many employees are social drinkers?
   SELECT COUNT(*) FROM absenteeism WHERE Social_drinker = 1;

6. How many employees are social smokers?
   SELECT COUNT(*) FROM absenteeism WHERE Social_smoker = 1;


Medium Questions:
7. On which days of the week does the average absenteeism time exceed 4 hours?
   SELECT Day_of_the_week
   FROM absenteeism
   GROUP BY Day_of_the_week
   HAVING AVG(Absenteeism_time_in_hours) > 4;

8. What is the average transportation expense for each season?
   SELECT Seasons, AVG(Transportation_expense)
   FROM absenteeism
   GROUP BY Seasons;

9. Which reasons for absence are more frequent for social drinkers than social non-drinkers?
   SELECT Reason_for_absence
   FROM absenteeism
   WHERE Social_drinker = 1
   GROUP BY Reason_for_absence
   HAVING COUNT(*) > (SELECT COUNT(*) FROM absenteeism WHERE Social_drinker = 0 GROUP BY Reason_for_absence);

10. Calculate the summary absenteeism time per reason for absence.
    SELECT Reason_for_absence, SUM(Absenteeism_time_in_hours)
    FROM absenteeism
    GROUP BY Reason_for_absence;

11. What are the top 3 reasons for absence in each season?
    SELECT Seasons, Reason_for_absence, COUNT(*)
    FROM absenteeism
    GROUP BY Seasons, Reason_for_absence
    ORDER BY Seasons, COUNT(*) DESC
    LIMIT 3;

12. What is the average distance from residence to work for each education level?
    SELECT Education, AVG(Distance_from_Residence_to_Work)
    FROM absenteeism
    GROUP BY Education;


Hard Questions:
13. Find employees whose total absenteeism hours are above the average.
    SELECT Age
    FROM absenteeism
    GROUP BY Age
    HAVING SUM(Absenteeism_time_in_hours) > (SELECT AVG(Absenteeism_time_in_hours) FROM absenteeism);

14. What is the average absenteeism time for employees with BMI higher than the average BMI?
    SELECT AVG(Absenteeism_time_in_hours)
    FROM absenteeism
    WHERE Body_mass_index > (SELECT AVG(Body_mass_index) FROM absenteeism);

15. How many employees have a higher transportation expense than the average transportation expense and live closer than the average distance from work?
    SELECT COUNT(*)
    FROM absenteeism
    WHERE Transportation_expense > (SELECT AVG(Transportation_expense) FROM absenteeism)
    AND Distance_from_Residence_to_Work < (SELECT AVG(Distance_from_Residence_to_Work) FROM absenteeism);

16. Find the age of employees who have been absent for more than 5 hours due to reason 26.
    SELECT Age
    FROM absenteeism
    WHERE Reason_for_absence = 26
    AND Absenteeism_time_in_hours > 5;

17. Find the top 3 ages with the highest total absenteeism hours, excluding disciplinary failures.
    SELECT Age, SUM(Absenteeism_time_in_hours)
    FROM absenteeism
    WHERE Disciplinary_failure = 0
    GROUP BY Age
    ORDER BY SUM(Absenteeism_time_in_hours) DESC
    LIMIT 3;

18. Which months have a higher total absenteeism time than the previous month?
    SELECT Month_of_absence
    FROM absenteeism
    WHERE (SELECT SUM(Absenteeism_time_in_hours) FROM absenteeism WHERE Month_of_absence = t.Month_of_absence - 1)
          < (SELECT SUM(Absenteeism_time_in_hours) FROM absenteeism WHERE Month_of_absence = t.Month_of_absence)
          AS t;

Find the total weight of all absente workers in the company in the month of April:

In [27]:
%%sql 
SELECT SUM(weight)
FROM absenteeism
WHERE Month_of_absence = 4;

*  sqlite://
Done.


SUM(weight)
4176


Find the average number of hours of absenteeism for each reason:


In [28]:
%%sql
SELECT Reason_for_absence, AVG(Absenteeism_time_in_hours) AS Average_Absenteeism
FROM absenteeism
GROUP BY Reason_for_absence;

*  sqlite://
Done.


Reason_for_absence,Average_Absenteeism
0,0.0
1,11.375
2,24.0
3,8.0
4,4.5
5,6.333333333333333
6,21.375
7,10.0
8,5.333333333333333
9,42.0


Find the total number of disciplinary failures for each season:


In [29]:
%%sql
SELECT Seasons, COUNT(Disciplinary_failure) AS Total_Disciplinary_Failures
FROM absenteeism
GROUP BY Seasons;

*  sqlite://
Done.


Seasons,Total_Disciplinary_Failures
1,170
2,192
3,183
4,195


Find the average distance from residence to work for each category of education:

In [30]:
%%sql
SELECT Education, AVG(Distance_from_Residence_to_Work) AS Average_Distance
FROM absenteeism
GROUP BY Education;

*  sqlite://
Done.


Education,Average_Distance
1,31.378068739770868
2,23.608695652173914
3,20.367088607594937
4,15.0


Find the top 5 employees with the highest transportation expenses:

In [31]:
%%sql
SELECT *
FROM absenteeism
ORDER BY Transportation_expense DESC
LIMIT 5;

*  sqlite://
Done.


ID,Reason_for_absence,Month_of_absence,Day_of_the_week,Seasons,Transportation_expense,Distance_from_Residence_to_Work,Service_time,Age,Work_load_Average_per_day_,Hit_target,Disciplinary_failure,Education,Son,Social_drinker,Social_smoker,Pet,Weight,Height,Body_mass_index,Absenteeism_time_in_hours
31,11,2,2,2,388,15,9,50,302.585,99,0,1,0,0,0,0,76,178,24,8
31,1,2,3,2,388,15,9,50,302.585,99,0,1,0,0,0,0,76,178,24,8
31,0,5,4,3,388,15,9,50,378.884,92,1,1,0,0,0,0,76,178,24,0
23,19,4,4,3,378,49,11,36,326.452,96,0,1,2,0,1,4,65,174,21,8
23,0,5,4,3,378,49,11,36,378.884,92,1,1,2,0,1,4,65,174,21,0


### Custom queries defined by the user

In [33]:
data_query = "What day of the week has the most absenteeism?"

datagen_prompts_2 = [
    {"role" : "system", "content" : "You are a data analyst specializing in SQL, you are presented with a natural language query, and you form queries to answer questions about the data."},
    {"role" : "user", "content" : f"Please generate 1 SQL queries for data with columns {', '.join(df.columns)}. The table is called 'absenteeism'. Use the natural language query {data_query}"},
]

prompter.prompt_model_print(datagen_prompts_2)

SELECT Day_of_the_week, COUNT(*) as Absenteeism_count 
FROM absenteeism 
GROUP BY Day_of_the_week 
ORDER BY Absenteeism_count DESC 
LIMIT 1; 

This SQL query selects the Day_of_the_week column and counts the number of absenteeism entries associated with each day. It then groups them by day of the week, orders them by the absenteeism count in descending order, and limits the output to only show the day with the highest absenteeism count.

In [34]:
%%sql 
SELECT Day_of_the_week, COUNT(*) as Absenteeism_count 
FROM absenteeism 
GROUP BY Day_of_the_week 
ORDER BY Absenteeism_count DESC 
LIMIT 1;

*  sqlite://
Done.


Day_of_the_week,Absenteeism_count
2,161
